<a href="https://colab.research.google.com/github/bluebynick/CEE-9730-Data-Science-for-Geotechnical-Engineers/blob/main/assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Cell 1: install and imports, mount Drive
#!pip install -q tensorflow==2.15.0  # or a current TF2.x
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, glob, random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
## from tensorflow import keras
## from tensorflow import layers
from sklearn.metrics import confusion_matrix
print("TensorFlow version:", tf.__version__)

#get the files
leftImg8bit = os.path.join('/content/drive/MyDrive/leftImg8bit_trainvaltest', 'leftImg8bit')
gtCoarse = os.path.join('/content/drive/MyDrive/gtCoarse', 'gtCoarse')

#i'd tried with zip files but it ended horribly so i restarted lol

IMAGE_SIZE = (256, 512)   #(height, width) - reduce for speed.
BATCH_SIZE = 8
EPOCHS = 20               #how many epochs per LR run (reduced for quick debug)
N_CLASSES = 12            #number of target classes
IGNORE_LABEL = 255

#limit number of files
DEBUG_LIMIT_TRAIN = None   #set to None to use all
DEBUG_LIMIT_VAL = None

print("Paths set. leftImg8bit exists?", os.path.exists(leftImg8bit))
print("gtCoarse exists?", os.path.exists(gtCoarse))

#a method which catches the image and pairs it to the label
def find_image_label_pairs(split='train'):
    #define variables per instance
    labels = {}
    pairs = []

    #for the images
    img_pattern = os.path.join(leftImg8bit, split, '*', '*_leftImg8bit.png') #create a list per the image handle
    imgs = sorted(glob.glob(img_pattern)) #create a list of images matching the pattern that we established
    #glob.glob returns a list of paths matching a pathname pattern - from the docs

    #for the labels
    #there's two possible label naming conventions so we have to parse through all in a loop
    label_pattern1 = os.path.join(gtCoarse, split, '*', '*_gtCoarse_labelTrainIds.png')
    label_pattern2 = os.path.join(gtCoarse, split, '*', '*_gtCoarse_labelIds.png')

    #go through all of #1, get it, assign label to the index, replace it with null in the ram space
    #then do it to #2
    for p in glob.glob(label_pattern1):
        labels[Path(p).stem.replace('_gtCoarse_labelTrainIds','')] = p
    for p in glob.glob(label_pattern2):
        labels[Path(p).stem.replace('_gtCoarse_labelIds','')] = p

    #per each img, get it and replace it's ram spot in ram with null
    #but get the label associated at that location. then put them together if the label matches the image
    for img in imgs:
        stem = Path(img).stem.replace('_leftImg8bit','')
        lab = labels.get(stem)
        if lab:
            pairs.append((img, lab))

    #pass me back the array of pairs
    return pairs

#call the method, passing it the different folders we have. idk if i should pass test / train_extra too tho
train_pairs = find_image_label_pairs('train')
val_pairs = find_image_label_pairs('val')
print("Found pairs:", len(train_pairs), "train |", len(val_pairs), "val")

#reduce using limits for a quick run
if DEBUG_LIMIT_TRAIN:
    train_pairs = train_pairs[:DEBUG_LIMIT_TRAIN]
if DEBUG_LIMIT_VAL:
    val_pairs = val_pairs[:DEBUG_LIMIT_VAL]
print("Using pairs:", len(train_pairs), "train |", len(val_pairs), "val")

#this was a test
#for t in train_pairs[:5]:
#    print(t[0], "\n", t[1], "\n")



Mounted at /content/drive
TensorFlow version: 2.19.0
Paths set. leftImg8bit exists? True
gtCoarse exists? True
Found pairs: 2500 train | 500 val
Using pairs: 2500 train | 500 val
